In [2]:
import numpy as np
import pandas as pd
import gzip

In [33]:
df = pd.read_csv('hosp/diagnoses_icd.csv.gz')
pt_icu = pd.read_csv('icu/icustays.csv.gz')
df_lab = pd.read_csv('hosp/labevents.csv.gz')

# HF ICD without BNP

In [10]:
hfref_id = df[df['icd_code'].str.startswith('4282')|df['icd_code'].str.startswith('I502')].subject_id

In [11]:
hfpef_id = df[df['icd_code'].str.startswith('4283')|df['icd_code'].str.startswith('I503')].subject_id

In [15]:
hf_id = pd.concat([hfref_id, hfpef_id])

In [19]:
hf_id

252        10000980
308        10000980
330        10000980
355        10000980
1180       10002155
             ...   
4755413    19998330
4755429    19998330
4755446    19998330
4755463    19998330
4755736    19998591
Name: subject_id, Length: 49725, dtype: int64

In [18]:
len(hf_id)

49725

In [24]:
icu_id = pt_icu.subject_id
len(pt_icu.subject_id.unique())

50920

In [32]:
icu_hf_pt = pt_icu[pt_icu["subject_id"].isin(hf_id)]
icu_hf_pt

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
6,10002013,23581541,39060235,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352
7,10002155,20345487,32358465,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,2131-03-10 18:09:21,0.858576
8,10002155,23822395,33685454,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912
9,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447
...,...,...,...,...,...,...,...,...
73163,19998330,24096838,31199714,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2178-11-29 21:51:19,2178-11-29 22:44:27,0.036898
73164,19998330,24096838,33428243,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2178-11-27 22:53:00,2178-11-29 21:29:39,1.942118
73165,19998330,24492004,32641669,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2178-10-01 08:51:00,2178-10-03 23:25:08,2.607037
73166,19998591,24349193,31144045,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2185-07-16 18:48:18,2185-07-26 18:27:01,9.985220


In [31]:
len(icu_hf_pt["subject_id"].unique())

12426

In [34]:
df_lab_id = pd.read_csv('hosp/d_labitems.csv.gz')

In [35]:
df_lab_comp = pd.merge(df_lab, df_lab_id, on='itemid', how='left')

In [36]:
pt_with_pro_bnp = df_lab_comp[(df_lab_comp['itemid'] == 50963)]

In [37]:
pt_with_pro_bnp.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments,label,fluid,category
5090,5099,10000935,25849114.0,55494709,50963,NaN,2187-10-11 06:25:00,2187-10-11 18:35:00,___,617.0,pg/mL,0.0,226.0,abnormal,ROUTINE,"REFERENCE VALUES VARY WITH AGE, SEX, AND RENAL...",NTproBNP,Blood,Chemistry
5917,5927,10000980,NaN,33821244,50963,NaN,2188-01-03 13:18:00,2188-01-03 14:42:00,___,4571.0,pg/mL,0.0,624.0,abnormal,STAT,"REFERENCE VALUES VARY WITH AGE, SEX, AND RENAL...",NTproBNP,Blood,Chemistry
6206,6216,10000980,NaN,46753637,50963,NaN,2189-06-27 06:48:00,2189-06-27 07:56:00,___,2826.0,pg/mL,0.0,624.0,abnormal,STAT,"REFERENCE VALUES VARY WITH AGE, SEX, AND RENAL...",NTproBNP,Blood,Chemistry
6891,6902,10000980,NaN,88477575,50963,NaN,2190-11-06 16:00:00,2190-11-06 18:12:00,___,4279.0,pg/mL,0.0,624.0,abnormal,STAT,"REFERENCE VALUES VARY WITH AGE, SEX, AND RENAL...",NTproBNP,Blood,Chemistry
7582,7593,10000980,NaN,77976228,50963,NaN,2191-05-30 12:40:00,2191-05-30 13:55:00,___,6440.0,pg/mL,0.0,624.0,abnormal,STAT,"REFERENCE VALUES VARY WITH AGE, SEX, AND RENAL...",NTproBNP,Blood,Chemistry


In [70]:
icu_bnp = pt_icu[pt_icu["subject_id"].isin(pt_with_pro_bnp["subject_id"])]

In [71]:
len(icu_bnp["subject_id"].unique())

14538

In [72]:
len(icu_bnp[(icu_bnp["subject_id"].isin(icu_hf_pt["subject_id"]))]["subject_id"].unique())

8211

In [73]:
len(icu_bnp[~(icu_bnp["subject_id"].isin(icu_hf_pt["subject_id"]))]["subject_id"].unique())

6327

In [41]:
# Using isin() to create a boolean mask where subject_id is present in pt_with_pro_bnp
is_in_pro_bnp = icu_hf_pt["subject_id"].isin(pt_with_pro_bnp["subject_id"])

# Using ~ to negate the boolean mask, giving us True where subject_id is NOT in pt_with_pro_bnp
not_in_pro_bnp = ~is_in_pro_bnp

# Selecting rows from icu_hf_pt where subject_id is NOT in pt_with_pro_bnp
hf_pt_without_bnp = icu_hf_pt[not_in_pro_bnp]

hf_pt_with_bnp = icu_hf_pt[is_in_pro_bnp]

In [44]:
print(len(hf_pt_without_bnp.subject_id.unique()))
print(len(hf_pt_with_bnp.subject_id.unique()))
print(len(icu_hf_pt.subject_id.unique()))


4215
8211
12426


In [69]:
print(len(hf_pt_without_bnp[hf_pt_without_bnp["subject_id"].isin(hfref_id)].subject_id.unique())) 
print(len(hf_pt_without_bnp[hf_pt_without_bnp["subject_id"].isin(hfpef_id)].subject_id.unique()))
print(len(hf_pt_without_bnp[hf_pt_without_bnp["subject_id"].isin(hfpef_id)|hf_pt_without_bnp["subject_id"].isin(hfref_id)].subject_id.unique()))
print(len(hf_pt_without_bnp[hf_pt_without_bnp["subject_id"].isin(hfpef_id)&hf_pt_without_bnp["subject_id"].isin(hfref_id)].subject_id.unique()))

2351
2041
4215
177


# Returned PT

In [53]:
icu_hf_pt["hadm_id"].value_counts()

20084622    7
28682905    6
23006176    6
25332191    6
20173617    6
           ..
28310229    1
23105989    1
27051245    1
28383961    1
21923841    1
Name: hadm_id, Length: 20133, dtype: int64

In [56]:
# Use the duplicated() method to create a boolean mask of duplicate "hadm_id"
is_duplicate_hadm_id = icu_hf_pt["hadm_id"].duplicated(keep=False)

# Use boolean indexing to filter the DataFrame based on the duplicate condition
result_df = icu_hf_pt[is_duplicate_hadm_id]

In [58]:
len(result_df["subject_id"].unique())

2085

In [85]:
bnp_without_icd = icu_bnp[~(icu_bnp["subject_id"].isin(icu_hf_pt["subject_id"]))]["hadm_id"].duplicated(keep=False)

len(bnp_without_icd)

10581